In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


In [2]:
dataset = pd.read_csv('/content/housing.csv')   # path to your csv file
# drop any incomplete enteries
dataset = dataset.dropna()
#iew the first five rows of the dataset
print(" here are the first five rows")
dataset.head(10)

 here are the first five rows


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


# New Section

In [0]:
#Next we will split the report dataset into input x and output Y
Y = dataset['median_house_value']  # we will predict the median house value column
# the remainder of the coumns will be used to predict Y
# Select from the  longitude column to the 'median_income' column
X= dataset.loc[:,'longitude':'median_income']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state = 2003)
x_train_np = x_train.to_numpy()
y_train_np = y_train.to_numpy()
#convert the testing data
x_test_np = x_test.to_numpy()
y_test_np = y_test.to_numpy()

In [0]:
import torch

from torch.nn import Conv1d #convolution layer

from torch.nn import MaxPool1d # max pooling layer

from torch.nn import Flatten #flatten layer

from torch.nn import Linear  #linear layer

from torch.nn.functional import relu #reLU activation function

from torch.utils.data import DataLoader, TensorDataset 


In [0]:
class CnnRegressor(torch.nn.Module):
  #define the initialization method
  def __init__(self, batch_size, inputs, outputs):
    # initialize the superclass and store the paramteers
    super(CnnRegressor, self).__init__()
    self.batch_size = batch_size
    self.inputs = inputs
    print(self.inputs)
    self.outputs = outputs

    #define the input layer
    #( input channels, output channels, kernel size)
    self.input_layer = Conv1d(inputs, batch_size, 1)
    
    # Define a max pooling layer
    self.conv_layer = Conv1d(batch_size, 256, 1)
    self.max_pooling_layer = MaxPool1d(1)

    self.conv_layer2 = Conv1d(256, 128, 1)
    
    self.max_pooling_layer2 = MaxPool1d(1)

    
    #  Adding second convolution layer
    self.conv_layer3 = Conv1d(128,64,1)
    
    #self.max_pooling_layer3 = MaxPool1d(1)
    
    self.flatten_layer = Flatten()

    self.linear_layer = Linear(64, 32)

    self.output_layer = Linear(32, outputs)


    

  
  
  def feed(self, input):
    input = input.reshape((self.batch_size, self.inputs, 1))

    output = relu(self.input_layer(input))

    output = self.max_pooling_layer(output)
    
    output = relu(self.conv_layer(output))

    output = self.max_pooling_layer(output)
    
    output = relu(self.conv_layer2(output))
    
    output = self.max_pooling_layer2(output)
    
    output = relu(self.conv_layer3(output))
    
    #output = self.max_pooling_layer3(output)

    output = self.flatten_layer(output)

    output = self.linear_layer(output)
    
    output = self.output_layer(output)


    return output



In [7]:
from torch.optim import SGD
from torch.nn import L1Loss
!pip install pytorch-ignite
from ignite.contrib.metrics.regression.r2_score import R2Score

In [8]:
batch_size = 128
# (batch_size, X columns, Y columns)
model = CnnRegressor(batch_size, X.shape[1], 1)
#set the model to use the cpu for processing
model.cuda()

8


CnnRegressor(
  (input_layer): Conv1d(8, 128, kernel_size=(1,), stride=(1,))
  (conv_layer): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
  (max_pooling_layer): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer2): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
  (max_pooling_layer2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer3): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
  (flatten_layer): Flatten()
  (linear_layer): Linear(in_features=64, out_features=32, bias=True)
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
)

In [0]:
def model_loss(model, dataset, train = False, optimizer = True):
  performance = L1Loss()
  score_metric = R2Score()

  avg_loss = 0
  avg_score = 0
  count = 0

  for input, output in iter(dataset):
    predictions = model.feed(input)

    loss = performance(predictions, output)
    score_metric.update([predictions, output])
    score = score_metric.compute()

    if(train):
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    
    avg_loss += loss.item()
    avg_score += score
    count += 1
  return avg_loss/count, avg_score/count

In [0]:
epochs = 10000

optimizer = SGD(model.parameters(), lr=1e-6)

inputs = torch.from_numpy(x_train_np).cuda().float()

outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0], 1)).cuda().float()
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)
for epoch in range(epochs):
  avg_loss, avg_r2_score = model_loss(model, loader, train=True, optimizer=optimizer)
  print("Epoch "+ str(epoch +1) + ":\n\tloss = "+ str(avg_loss) + "\n \tR^2 Score = "+ str(avg_r2_score))

Streaming output truncated to the last 5000 lines.
 	R^2 Score = 0.5701023641421891
Epoch 4704:
	loss = 55567.48127815315
 	R^2 Score = 0.5451009933676524
Epoch 4705:
	loss = 56490.70365287162
 	R^2 Score = 0.5557389328675106
Epoch 4706:
	loss = 55779.50179476351
 	R^2 Score = 0.5767734484365838
Epoch 4707:
	loss = 55499.95755912162
 	R^2 Score = 0.563807357257125
Epoch 4708:
	loss = 55642.83146818694
 	R^2 Score = 0.5551209512775727
Epoch 4709:
	loss = 55859.02857545045
 	R^2 Score = 0.5387538502535845
Epoch 4710:
	loss = 55545.44636824324
 	R^2 Score = 0.5552494407818672
Epoch 4711:
	loss = 55319.71055039414
 	R^2 Score = 0.5712593715783005
Epoch 4712:
	loss = 56840.13942849099
 	R^2 Score = 0.5326807930732511
Epoch 4713:
	loss = 54830.60571509009
 	R^2 Score = 0.5816358016024539
Epoch 4714:
	loss = 56053.81809543919
 	R^2 Score = 0.5555802155838817
Epoch 4715:
	loss = 55422.63302364865
 	R^2 Score = 0.5660829689898959
Epoch 4716:
	loss = 55991.41321790541
 	R^2 Score = 0.56296575578

In [0]:
torch.save(model,'/content/model')

In [0]:
inputs = torch.from_numpy(x_test_np).cuda().float()
outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0],1)).cuda().float()
tensor = TensorDataset(inputs,outputs)
loader = DataLoader(tensor, batch_size, shuffle=True,drop_last=True)

avg_loss, avg_r2_score = model_loss(model, loader)
print( " the model's l1 loss is:"+ str(avg_loss))
print("the models avg r^2 score is:" +str(avg_r2_score))

In [0]:
torch.save(model.state_dict(), 'saved_model.pt')

In [0]:
a=model.load_state_dict(torch.load('saved_model.pt'))
print(a)